# Load T5-base QA (t5-base-finetuned-question-answering) Model and predict on validation data fro SQuAD Question-answering Dataset


### Code Reference: 

https://huggingface.co/MaRiOrOsSi/t5-base-finetuned-question-answering

### 1. Import Packages relevant ot modelling

In [1]:
from  transformers  import  AutoTokenizer, AutoModelWithLMHead, pipeline
import pandas as pd

### 2. Load model and tokenizer

In [2]:
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
model = AutoModelWithLMHead.from_pretrained(model_name)

C:\Users\basharm\AppData\Local\anaconda3\Lib\site-packages\transformers\models\auto\modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


### 3. Test pretrained model

In [5]:
question = "Who graduated from QUT?"
context = "The name of the scientist is Dr. Bashar. He graduated from QUT."
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

Dr. Bashar


C:\Users\basharm\AppData\Local\anaconda3\Lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### 4. Import packages relevant to data loading

In [ ]:
from datasets import load_dataset

### 5. Load Dataset

In [6]:
squad = load_dataset("squad")

Found cached dataset parquet (C:/Users/basharm/.cache/huggingface/datasets/parquet/plain_text-d0ce9b3222e19e32/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
squad['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

### 7. Define predict funstion

In [11]:
def predict_answer(question, context):
    input = f"question: {question} context: {context}"
    encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
    output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return output

### 8. Make prediction on validation data

In [ ]:
count = 0
out_rows = []
for row in squad['validation']:
    question = row['question']
    #print(question)
    context = row['context']
    #print(context)
    ans = row['answers']['text'][0]
    #print(ans)
    pred_ans = predict_answer(question, context)
    #print(pred_ans)
    out_row = {'answer': ans, 'pred_ans': pred_ans}
    out_rows.append(out_row)
    count += 1
    if count%100==0:
        print('Completed Respon Count', count)
    
df_out_val = pd.DataFrame(out_rows)

In [9]:
df_out_val

,answer,pred_ans
0,Denver Broncos,Denver Broncos
1,Carolina Panthers,Denver Broncos
2,"Santa Clara, California","Santa Clara, California"
3,Denver Broncos,Denver Broncos
4,gold,gold
...,...,...
10565,kilogram-force,Kilowatt-force
10566,kilopond,Kilopond
10567,slug,metric slug
10568,kip,Kilowatt-force


### 9. Stro predicted asnwers

In [10]:
df_out_val.to_csv('T5_squad_valid.csv', index=None)

### 10. Devine evaluation functions and evaluate the prediction accuracy

In [11]:
def exact_match(pred_tokens, true_tokens):
    '''
    A straightforward way to check the equality of the two lists in Python 
    is by using the equality == operator. 
    When the equality == is used on the list type in Python, 
    it returns True if the lists are equal and False if they are not.
    '''
    return int(pred_tokens==true_tokens)

def half_exact_match(pred_tokens, true_tokens):
    '''
    A straightforward way to check the equality of the two lists in Python 
    is by using the equality == operator. 
    When the equality == is used on the list type in Python, 
    it returns True if the lists are equal and False if they are not.
    '''
    if len(pred_tokens)<=1 or len(true_tokens)<=1:
        return int(pred_tokens==true_tokens) 
    else:
        return int(pred_tokens[0]==true_tokens[0] or pred_tokens[-1]==true_tokens[-1])
    
def any_token_match(pred_tokens, true_tokens):
    common_tokens = set(pred_tokens) & set(true_tokens)
    return int(len(common_tokens)>0)
    

def get_prec_rec_f1(pred_tokens, true_tokens):
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(true_tokens) == 0:
        prec = rec = f1 = 1
        return prec, rec, f1
    
    common_tokens = set(pred_tokens) & set(true_tokens)
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        prec = rec = f1 = 0
        return prec, rec, f1
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(true_tokens)
    f1 = 2 * (prec * rec) / (prec + rec)
    
    return prec, rec, f1

In [13]:
match = 0
half_match = 0
any_match = 0
prec = 0
rec = 0
f1 = 0
count = 0
for idx in df_out_val.index:

    pred_tokens = df_out_val.loc[idx, 'pred_ans'].split()
    true_tokens = df_out_val.loc[idx, 'answer'].split()
    
    match +=  exact_match(pred_tokens, true_tokens)
    half_match += half_exact_match(pred_tokens, true_tokens)
    any_match += any_token_match(pred_tokens, true_tokens)
    
    score =  get_prec_rec_f1(pred_tokens, true_tokens)
    prec += score[0]
    rec += score[1]
    f1 += score[2]
    count += 1

    
import datetime
now = datetime.datetime.now()
string = ''
string += '========={}========\n'.format(now)
string += '========={}========\n'.format(now)
string += 'exact_match: '+str(match/count)+'\n'
string += 'half_exact_match: '+str(half_match/count)+'\n'
string += 'any_match: '+str(any_match/count)+'\n'
string += 'recall: '+str(rec/count)+'\n'
string += 'precision: '+str(prec/count)+'\n'
string += 'f1: '+str(f1/count)+'\n\n'
print(string)

with open('Report_T5_Squad.txt', 'a+') as FO:
    FO.write(string)

=========2025-02-21 14:52:02.533026========
=========2025-02-21 14:52:02.533026========
exact_match: 0.38420056764427624
half_exact_match: 0.584484389782403
any_match: 0.7371807000946073
recall: 0.6144846712364499
precision: 0.6243597824517078
f1: 0.6001928249863442


